In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess

from researchgate.researchgate.spiders.paper import PaperSpider

In [3]:
process = CrawlerProcess(
    settings={
        "FEEDS": {"items.json": {"format": "json"}},
        "LOG_ENABLED": False
        #     "LOG_LEVEL": 'INFO',
    }
)
process.crawl(PaperSpider)
process.start()

In [2]:
import json

with open("items.json") as f:
    items = json.load(f)

In [3]:
items[0]

{'id': '323694313',
 'title': 'The Lottery Ticket Hypothesis: Training Pruned Neural Networks',
 'abstract': 'Recent work on neural network pruning indicates that, at training time, neural networks need to be significantly larger in size than is necessary to represent the eventual functions that they learn. This paper articulates a new hypothesis to explain this phenomenon. This conjecture, which we term the "lottery ticket hypothesis," proposes that successful training depends on lucky random initialization of a smaller subcomponent of the network. Larger networks have more of these "lottery tickets," meaning they are more likely to luck out with a subcomponent initialized in a configuration amenable to successful optimization. This paper conducts a series of experiments with XOR and MNIST that support the lottery ticket hypothesis. In particular, we identify these fortuitously-initialized subcomponents by pruning low-magnitude weights from trained networks. We then demonstrate that t

## Elasticsearch 

In [4]:
from elasticsearch import Elasticsearch

### Connect 

In [5]:
es = Elasticsearch(hosts=[{"host": "localhost", "port": 9200}])

### Clear previous index 

In [6]:
es.indices.delete("paper-index", ignore=404)

{'acknowledged': True}

### Create new index 

In [7]:
es.indices.create(index="paper-index", ignore=400)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'paper-index'}

### Create persistent layer 

In [8]:
from elasticsearch_dsl import (
    Document,
    Date,
    Nested,
    Boolean,
    analyzer,
    InnerDoc,
    Completion,
    Keyword,
    Text,
    Integer,
    Float,
)

In [9]:
class Paper(Document):
    title = Text(fields={"raw": Keyword()})
    date = Integer()
    abstract = Text()
    authors = Text()
    references = Text()
    page_rank = Float()

    class Index:
        name = "paper-index"

In [10]:
# create the mappings in Elasticsearch
Paper.init(using=es)

### Insert items 

In [11]:
for item in items:
    paper = Paper(meta={"id": item["id"]}, **item)
    paper.save(using=es)

### Funcions for inserting items and clearing index

In [65]:
def gen_utils(host = None):
    if host is None:
        host = {"host": "localhost", "port": 9200}
    es = Elasticsearch(hosts=[{"host": "localhost", "port": 9200}])
    def clear_index():
        es.indices.delete("paper-index", ignore=404)
        es.indices.create(index="paper-index", ignore=400)
    def insert_items(items):
        for item in items:
            paper = Paper(meta={"id": item["id"]}, **item)
            paper.save(using=es)
    return clear_index, insert_items

In [66]:
clear, insert = gen_utils()

In [67]:
clear()

In [68]:
insert(items)

## Calculating page rank

In [69]:
import numpy as np

In [70]:
# by calling .search we get back a standard Search object
s = Paper.search(using=es)

In [71]:
res = s.execute()

In [72]:
all_papers = list(res)

In [73]:
all_ids = sorted(list(map(lambda x: x.id, all_papers)))

In [74]:
p_matrix = np.zeros((len(all_ids), len(all_ids)))

In [76]:
id_loc = dict()
for index, paper_id in enumerate(all_ids):
    id_loc[paper_id] = index

In [77]:
for index, paper_id in enumerate(all_ids):
    paper = Paper.get(id=paper_id, using=es)
    for reference_id in paper.references:
        try:
            p_matrix[index, id_loc[reference_id]] = 1
        except KeyError:
            continue

In [78]:
alpha = 0.1
N = len(all_ids)
v = np.ones((1, N))

In [79]:
row_sums = np.sum(p_matrix, axis=1, keepdims=True)

In [80]:
# first part is for rows having nonzero elements
# second part is for dead-ends
p_matrix = ((row_sums > 0) * 1) * (
    (1 - alpha) * p_matrix / (row_sums + np.logical_not(row_sums > 0) * 1)
    + alpha * v / N
) + (np.logical_not(row_sums > 0) * 1) * v / N

In [81]:
p_matrix

array([[0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 ],
       [0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 ],
       [0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 ],
       [0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 ],
       [0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 ],
       [0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 ],
       [0.01, 0.01, 0.91, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
       [0.16, 0.16, 0.16, 0.01, 0.16, 0.01, 0.16, 0.01, 0.01, 0.16],
       [0.01, 0.01, 0.01, 0.46, 0.01, 0.46, 0.01, 0.01, 0.01, 0.01],
       [0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 ]])

In [82]:
x0 = np.ones((1, N)) / N

In [83]:
while True:
    next_state = x0 @ p_matrix
    if np.allclose(next_state, x0, rtol=0.0001):
        break
    x0 = next_state

In [84]:
next_state

array([[0.08959866, 0.08959866, 0.17023796, 0.11297278, 0.08959866,
        0.11297278, 0.08959866, 0.07791159, 0.07791159, 0.08959866]])

In [85]:
for index, paper_id in enumerate(all_ids):
    paper = Paper.get(id=paper_id, using=es)
    paper.update(page_rank = next_state[0, index], using=es)

In [15]:
from elasticsearch_dsl import Search

In [16]:
s = Search(using=es, index="paper-index").query("match", title="the lottery")

In [132]:
s = Search(using=es, index="paper-index").query("match", id="323694313")

In [17]:
response = s.execute()

In [18]:
for hit in response:
    print(hit.meta.score, hit.title)

5.3779216 The Lottery Ticket Hypothesis: Training Pruned Neural Networks


In [50]:
# by calling .search we get back a standard Search object
s = Paper.search(using=es)
# the search is already limited to the index and doc_type of our document
s = s.query("match", title="the lottery")

In [51]:
response = s.execute()

In [52]:
for hit in response:
    print(hit.meta.score, hit.title)

5.328736 The Lottery Ticket Hypothesis: Training Pruned Neural Networks


In [53]:
res = es.get(index="paper-index", id="323694313")

In [54]:
res

{'_index': 'paper-index',
 '_type': '_doc',
 '_id': '323694313',
 '_version': 2,
 '_seq_no': 30,
 '_primary_term': 1,
 'found': True,
 '_source': {'id': '323694313',
  'title': 'The Lottery Ticket Hypothesis: Training Pruned Neural Networks',
  'abstract': 'Recent work on neural network pruning indicates that, at training time, neural networks need to be significantly larger in size than is necessary to represent the eventual functions that they learn. This paper articulates a new hypothesis to explain this phenomenon. This conjecture, which we term the "lottery ticket hypothesis," proposes that successful training depends on lucky random initialization of a smaller subcomponent of the network. Larger networks have more of these "lottery tickets," meaning they are more likely to luck out with a subcomponent initialized in a configuration amenable to successful optimization. This paper conducts a series of experiments with XOR and MNIST that support the lottery ticket hypothesis. In par